In [1]:
import gensim.downloader as api
#model = api.load('word2vec-google-news-300')

In [1]:
from gensim.models import KeyedVectors


In [2]:
model2 = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300-SLIM.bin.gz', binary=True)

In [3]:
import pandas as pd
import nltk
from gensim.similarities import WmdSimilarity
import pickle
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement
from cassandra import ConsistencyLevel
cluster = Cluster(['127.0.0.1'])#Cluster(['199.60.17.32','199.60.17.65'])
session = cluster.connect('mcanute')

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/matt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
from nltk import download
stop_words = stopwords.words('english')
import gensim

In [5]:
from nltk import word_tokenize
nltk.download('punkt')  # Download data for tokenizer.

def preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

[nltk_data] Downloading package punkt to /home/matt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
from pyemd import emd

In [7]:
data = pd.read_csv('websites/ParsedWebsitesCleanSample.csv')

In [15]:
query = "SELECT * from Blogs;"
data = pd.DataFrame(list(session.execute(query)))

In [16]:
data = data.dropna() # !!!

In [17]:
data = data[data['body_text'].apply(len)>2] ## !!!

In [19]:
documents = data.values

In [20]:
wmd_corpus = data['body_text'].apply(preprocess).values

In [21]:
instance = WmdSimilarity(wmd_corpus, model2, num_best=3)

In [22]:
pickle.dump(instance,open('WordMoverDistance-BlogInstanceSLIM.pkl','wb'))

In [12]:
instance = pickle.load(open('WordMoverDistance-Instance.pkl','rb'))

In [23]:
import re

In [24]:
def regex_text(val):
    if val:
        reg = re.compile(r'\b\w{3,20}\b')
        temp = reg.findall(val)
        temp_set = set(temp)
        temp_list = list(temp_set)
        if len(temp_list) < 20:
            temp2 = temp_list
        elif len(temp_list) < 50:
            temp2 = temp_list[10:]
        elif len(temp_list) < 100:
            temp2 = temp_list[30:]
        else:
            temp2 = temp_list[50:100]
        result = " ".join(temp2) 
        return ("..." + result + "....")
    else:
        return "Empty string found"

In [30]:
data.head(1)

,website,back_color,body_font,body_text,last_update,link_color,text_color,title_font,title_text
0,https://lebleurenard.tumblr.com/,#4ad791,Helvetica Neue,Please send me asks ❤️ Moodboard for nights a...,2019-12-03 21:34:55,#A77DC2,#444444,Clarendon Text Pro,


In [34]:
def similarityQuery(description):  # !!!
    description = description.replace('data', '')
    query = preprocess(description)
    sims = instance[query]  # A query is simply a "look-up" in the similarity class.

    # Print the query and the retrieved documents, together with their similarities.
    results = []
    print(description)
    for i in range(3):

        row = []
        row.append(documents[sims[i][0]][0]) # Website
        row.append(documents[sims[i][0]][1]) # Back_Color
        row.append(documents[sims[i][0]][5]) # Link_Color
        row.append(documents[sims[i][0]][6]) # Text_Color
        row.append(documents[sims[i][0]][7]) # Title_Font
        row.append(documents[sims[i][0]][2]) # Body_Font
        
        row.append(documents[sims[i][0]][3]) # Text
        
        row.append(sims[i][1])    # Similarity


        results.append(row)
    resultsDF = pd.DataFrame(results)

    resultsDF.columns = ['Website','BackColor','LinkColor','TextColor','TitleFont','BodyFont','Text','Similarity']
    #resultsDF['Fonts'] = resultsDF['Fonts'].replace({'\" \'': '\", \'', "\' \'": "\', \'", '(\r\n)': ','}, regex=True)
    #resultsDF['Colours'] = resultsDF['Colours'].replace({' ': ', '}, regex=True)
    resultsDF['Text'] = resultsDF['Text'].apply(regex_text)
    resultsDF = resultsDF.sort_values(by=['Similarity'],ascending=False)
    return resultsDF.style.set_properties(subset=['Text'], **{'width': '350px'})

In [35]:
# similarityQuery('I will talk about some cool topics in programming')
similarityQuery('I like anime')

I like anime


,Website,BackColor,LinkColor,TextColor,TitleFont,BodyFont,Text,Similarity
0,https://residentevil2remake.tumblr.com/,#000000,#992727,#FFFFFF,Garamond Classic FS,Helvetica Neue,...but are not they how same carlos with don like the jill looks good bright....,0.53062
1,https://mwagneto.tumblr.com/,#050505,#60c7ff,#60c7ff,Calluna,Helvetica Neue,...but kinda Tulio asshole throwing had emotionally feel like ngl Miguel....,0.527802
2,https://jaboncito.tumblr.com/,#ffffff,#ff8d8d,#ff8d8d,Typewriter FS,Helvetica Neue,...quick post think doodles here couple forgot redra them like about dgs....,0.50471


In [27]:
import ast
def generateResult(description):
    df = similarityQuery(description)
    dict = {}
    aF = [ast.literal_eval(df.iloc[0]['Fonts']), ast.literal_eval(df.iloc[1]['Fonts']), ast.literal_eval(df.iloc[2]['Fonts'])]
    aC = [ast.literal_eval(df.iloc[0]['Colours']), ast.literal_eval(df.iloc[1]['Colours']), ast.literal_eval(df.iloc[2]['Colours'])]
    aT = [df.iloc[0]['Text'], df.iloc[1]['Text'], df.iloc[2]['Text']]
    
    font_size = []
    for i in range(len(aF)):
        font_size.append(len(aF[i]))
        for j in range(len(aF[i])):
            aF[i][j][1] = int(aF[i][j][1])
    
    color_size = []
    for i in range(len(aC)):
        color_size.append(len(aC[i]))
        for j in range(len(aC[i])):
            if (' ' in aC[i][j][0]) or (',' in aC[i][j][0]):
                aC[i][j][0] = aC[i][j][0].replace(' ','')
                aC[i][j][0] = aC[i][j][0].replace(',','')
    
    dict['f_size'] = font_size
    dict['c_size'] = color_size
    dict['fonts'] = aF
    dict['colors'] = aC
    dict['texts'] = aT
    return dict

In [28]:
generateResult("big data and machine learning")

big  and machine learning


ValueError: malformed node or string: Timestamp('2019-11-25 16:00:24')